In [1]:
import pandas as pd
import numpy as np
import os
import importlib
from datetime import datetime,date
import glob

In [2]:
# parentDir = os.path.abspath(os.path.join(os.getcwd(),'..'))
# os.chdir(parentDir)

# Check data_loader

## Check load_transaction

In [ ]:


def load_transaction(data_folder_path, transaction_file_pattern):
    transaction_files = _gather_transaction_files(data_folder_path,transaction_file_pattern)
    transactions = _combine_transaction_files(transaction_files)
    transactions = _clean_transactions(transactions)
    print(f"The latest transaction date is {transactions['Run Date'].max()}")
    return transactions


def _gather_transaction_files(data_folder_path,transaction_file_pattern):
    transaction_file_path_pattern = os.path.join(
        data_folder_path, transaction_file_pattern
    )
    transaction_files = glob.glob(transaction_file_path_pattern)
    return transaction_files

def _combine_transaction_files(transaction_files):
    transaction_list = [
        pd.read_csv(file, usecols=range(14)) for file in transaction_files
    ]
    transactions = pd.concat(transaction_list, ignore_index=True)
    return transactions

def _remove_NA_value(df,colName):
    df_copy = df.copy()
    df_copy = df_copy[
        df_copy[colName].notna()
    ] 
    return df_copy

def _remove_leading_space(df,colName):
    df_copy = df.copy()
    df_copy[colName] = df_copy[colName].str.lstrip()
    return df_copy

def _str_to_date(df, colName, format):
    df_copy = df.copy()
    df_copy[colName] = pd.to_datetime(
        df_copy[colName], format=format
    ).dt.date
    return df_copy

def _add_Transfer_symbol(df):
    df_copy = df.copy()
    df_copy.loc[df_copy["Symbol"] == "  ", "Symbol"] = "Transfer"
    return df_copy

def _sort_df_by_column(df, colName):
    df_copy = df.copy()
    df_copy = df_copy.sort_values(by=colName).reset_index(
        drop=True
    )
    return df_copy


def _clean_transactions(transactions):
    transactions = _remove_NA_value(transactions,"Amount ($)")
    transactions = _remove_leading_space(transactions,"Run Date")
    transactions = _str_to_date(transactions,"Run Date","%m/%d/%Y")
    transactions = _str_to_date(transactions,"Settlement Date","%m/%d/%Y")
    transactions = _add_Transfer_symbol(transactions)
    transactions = _remove_leading_space(transactions,"Symbol")
    transactions = _remove_leading_space(transactions,"Description")
    transactions = _sort_df_by_column(transactions,"Run Date")
    return transactions


In [4]:
data_folder_path = "./data"
transaction_file_pattern = "Accounts_History_*.csv"
position_file_pattern = "Portfolio_Positions_*.csv"

In [5]:
transaction_files = _gather_transaction_files(data_folder_path,transaction_file_pattern)
transactions = _combine_transaction_files(transaction_files)
transactions = _clean_transactions(transactions)

In [6]:
# transactions = _remove_NA_value(transactions,"Amount ($)")
# transactions = _remove_leading_space(transactions,"Run Date")
# transactions = _str_to_date(transactions,"Run Date","%m/%d/%Y")
# transactions = _str_to_date(transactions,"Settlement Date","%m/%d/%Y")
# transactions = _add_Transfer_symbol(transactions)
# transactions = _remove_leading_space(transactions,"Symbol")
# transactions = _sort_df_by_column(transactions,"Run Date")

## Check load_position

In [7]:

def load_position(data_folder_path, position_file_pattern):
    position_file = _gather_position_files(data_folder_path, position_file_pattern)

    if not position_file is None:
        position = pd.read_csv(position_file)
        position = _clean_position(position)
    else:
        print("No position file found.")
    return position

def _gather_position_files(data_folder_path, position_file_pattern):
    position_file_path_pattern = os.path.join(data_folder_path, position_file_pattern)
    position_files = glob.glob(position_file_path_pattern)
    position_file = _find_latest_position_file(position_files)
    return position_file


def _find_latest_position_file(position_files):
    latest_file = None
    latest_date = None

    for file_path in position_files:
        file_name = os.path.basename(file_path)
        date_str = file_name.split("_")[-1].replace(".csv", "")
        file_date = datetime.strptime(date_str, "%b-%d-%Y")

        if latest_date is None or file_date > latest_date:
            latest_date = file_date
            latest_file = file_path

    return latest_file


def _clean_position(position):
    position = _remove_NA_value(position,"Current Value")
    position = _transfer_dollar_to_float(position, "Current Value")
    position = _transfer_dollar_to_float(position, "Cost Basis Total")
    return position


def _transfer_dollar_to_float(df, colNames):
    """
    Change "$123,456" to 123456.0, and "--" to 0.0
    """
    df_copy = df.copy()
    # Replace any "--" with "$0"
    cleaned = df_copy[colNames].str.replace("--", "$0", regex=False)
    # Remove dollar sign and commas, then convert to float
    cleaned = cleaned.str.replace("$", "", regex=False).str.replace(",", "", regex=False)
    df_copy[colNames] = cleaned.astype(float)
    return df_copy


In [8]:
data_folder_path = "./data"
transaction_file_pattern = "Accounts_History_*.csv"
position_file_pattern = "Portfolio_Positions_*.csv"

In [9]:
position_file = _gather_position_files(data_folder_path, position_file_pattern)
position_file

'./data/Portfolio_Positions_Aug-05-2025.csv'

In [10]:
position = pd.read_csv(position_file)
position = _clean_position(position)

In [11]:
# position = _remove_NA_value(position,"Current Value")
# position = _transfer_dollar_to_float(position, "Current Value")
# position = _transfer_dollar_to_float(position, "Cost Basis Total")

# Check Portfolio

In [12]:
from scipy.optimize import newton
import numpy as np
def compute_irr(cashflows, dates, cob):
    """用 Newton-Raphson 方法计算 IRR（连续复利）"""
    def npv(r):
        return sum(cf * np.exp(-r * (cob - d).days / 365.0) for cf, d in zip(cashflows, dates))
    try:
        result = -newton(npv, 0.1)
    except RuntimeError:
        result = np.nan
    return result

def display_percentage(df, colNames: list):
    df_copy = df.copy()
    for col in colNames:
        df_copy[col] = df_copy[col].apply(lambda x: f"{x:.2%}")
    return df_copy

In [ ]:
class Portfolio:
    def __init__(self, transactions, position):
        self.transactions = transactions
        self.position = position
        self.cob = date.today()
        
        account_number_dic = {
            "Individual":'Z23390746',
            "401k":'86964',
            "HSA":'241802439',
            "Cash":'Z06872898'
        }
        
        self.individualTransactions = transactions[transactions['Account Number']==account_number_dic["Individual"]]
        self.individualPosition = position[position['Account Number']==account_number_dic["Individual"]]
        self.pensionTransactions = transactions[transactions['Account Number']==account_number_dic["401k"]]
        self.pensionPosition = position[position['Account Number']==account_number_dic["401k"]]
        self.HSATransactions = transactions[transactions['Account Number']==account_number_dic["HSA"]]
        self.HSAPosition = position[position['Account Number']==account_number_dic["HSA"]]
        self.cashTransactions = transactions[transactions['Account Number']==account_number_dic["Cash"]]
        self.cashPosition = position[position['Account Number']==account_number_dic["Cash"]]
        
        
        self.cashSymbols = ['FZFXX**','FZFXX']
        self.otherSymbols = ['Pending Activity']
        self.contributionSymbols = ['','Transfer']
        self.bondSymbols = self.get_bond_symbol_list()
        self.stockSymbols = self.get_stock_symbol_list()
        
    ## Main functions
    
    def get_individual_account_summary(self):
        bondSymbols = self.get_bond_symbol_list()
        bondTotalValue = self.get_total_symbols_value(bondSymbols)
        bondTotalIrr = self.get_combined_symbol_irr(bondSymbols)
        bondHoldingPeriod = self.get_combined_symbol_holding_period(bondSymbols)
        
        stockSymbols = self.get_stock_symbol_list()
        stockTotalValue = self.get_total_symbols_value(stockSymbols)
        stockTotalIrr = self.get_combined_symbol_irr(stockSymbols)
        stockHoldingPeriod = self.get_combined_symbol_holding_period(stockSymbols)
        
        cashSymbols = self.cashSymbols
        cashTotalValue = self.get_total_symbols_value(cashSymbols)
        cashTotalIrr = self.get_combined_symbol_irr(cashSymbols)
        cashHoldingPeriod = self.get_combined_symbol_holding_period(cashSymbols)
        
        totalIRR = self.get_total_irr()
        totalHoldingPeriod = self.get_combined_symbol_holding_period(cashSymbols+stockSymbols+bondSymbols)
        
        totalValue = self.individualPosition['Current Value'].sum()
        result = pd.DataFrame({
            'Type':['bond','stock','cash','Total'],
            'Value': [bondTotalValue,stockTotalValue,cashTotalValue,totalValue],
            'Percentage':[bondTotalValue,stockTotalValue,cashTotalValue,totalValue]/totalValue,
            'IRR':[bondTotalIrr, stockTotalIrr, cashTotalIrr,totalIRR],
            'Weighted Avg Holding Period':[bondHoldingPeriod,stockHoldingPeriod,cashHoldingPeriod,totalHoldingPeriod]
        } 
        )
        return result
    
        
    def get_all_stock_summary(self):
        stockSymbols = self.get_stock_symbol_list()
        currentValueResult = self.get_symbol_current_values(stockSymbols)
        irrResult = self.get_symbol_irrs(stockSymbols)
        holdingPeriodResult = self.get_symbol_holding_period(stockSymbols)
        result = pd.merge(currentValueResult, irrResult, on='Symbol')
        result = pd.merge(result, holdingPeriodResult, on='Symbol')
        result = result.sort_values(by='Current Value', ascending=False)
        return result
    
    def get_all_bond_summary(self,showExpiredBond=False):
        bondSymbols = self.get_bond_symbol_list()
        currentValueResult = self.get_symbol_current_values(bondSymbols)
        irrResult = self.get_symbol_irrs(bondSymbols)
        holdingPeriodResult = self.get_symbol_holding_period(bondSymbols)
        result = pd.merge(currentValueResult, irrResult, on='Symbol')
        result = pd.merge(result, holdingPeriodResult, on='Symbol')
        result = result.sort_values(by='Current Value', ascending=False)
        if not showExpiredBond:
            result = result[result['Current Value']>0]
        return result
    
    
    ## Help functions
    
    def get_combined_symbol_holding_period(self, listSymbols: list, unit = 30):
        subTransactions = self.transactions[self.transactions['Symbol'].isin(listSymbols)]
        buyTranactions = subTransactions[subTransactions['Amount ($)']<0]
        df = buyTranactions.copy()
        df['Days Held'] = (self.cob - df['Run Date']).apply(lambda x: x.days)
        df['Weight'] = df['Amount ($)'].abs()
        totalWeightedHold = (df['Days Held'] * df['Weight']).sum() / df['Weight'].sum()/ unit
        return totalWeightedHold
        
    def get_symbol_holding_period(self, listSymbols: list, unit = 30):
        subTransactions = self.transactions[self.transactions['Symbol'].isin(listSymbols)]
        buyTranactions = subTransactions[subTransactions['Amount ($)']<0]
        df = buyTranactions.copy()
        df['Days Held'] = (self.cob - df['Run Date']).apply(lambda x: x.days)
        df['Weight'] = df['Amount ($)'].abs()
        totalWeightedHold = (df['Days Held'] * df['Weight']).sum() / df['Weight'].sum()/ unit
        totalWeightedHoldRow = pd.DataFrame({'Symbol': ['Total'], 'Weighted Avg Holding Period': [totalWeightedHold]})

        # 分组计算加权平均
        weightedHold = (
            df.groupby('Symbol')
            .apply(lambda g: (g['Days Held'] * g['Weight']).sum() / g['Weight'].sum()/ unit, include_groups=False)
            .reset_index(name='Weighted Avg Holding Period')
        )
        
        weightedHold = pd.concat([weightedHold, totalWeightedHoldRow], ignore_index=True)
        return weightedHold
    
    
    def get_symbol_current_values(self, listSymbols: list):
        resultList = []
        totalCurrentValue = self.get_total_symbols_value(listSymbols)
        for symbol in listSymbols:
            currentValue = self.get_symbol_current_value(symbol)
            currentValuePercent = currentValue/totalCurrentValue
            resultList.append({
                'Symbol': symbol,
                'Current Value': currentValue,
                'Percentage': currentValuePercent
            })
        resultList.append({
                'Symbol': 'Total',
                'Current Value': totalCurrentValue,
                'Percentage': 1
            })
        return pd.DataFrame(resultList)
    
    def get_symbol_current_value(self,symbol):
        try:
            value = self.position.loc[self.position['Symbol']==symbol, 'Current Value'].values[0]
        except:
            value = 0
        return value
        
    def get_total_symbols_value(self, symbols: list):
        Position = self.individualPosition[self.individualPosition['Symbol'].isin(symbols)]
        totalValue = Position['Current Value'].sum()
        return totalValue
    
    def get_symbol_irrs(self, listSymbols: list):
        resultList = []
        for symbol in listSymbols:
            irr = self.get_combined_symbol_irr([symbol])
            resultList.append({
                'Symbol': symbol,
                'IRR': irr
            })
        totalIrr = self.get_combined_symbol_irr(listSymbols)
        resultList.append({
                'Symbol': 'Total',
                'IRR': totalIrr
            })
        return pd.DataFrame(resultList)
    
    
    def get_total_irr(self):
        trans = self.individualTransactions[self.individualTransactions['Symbol'].isin(self.contributionSymbols)]
        cashflows = trans['Amount ($)'].tolist()
        cashflows = [-x for x in cashflows]
        dates = trans['Run Date'].tolist()
        current_value = self.individualPosition['Current Value'].sum()
        cashflows.append(current_value)
        dates.append(self.cob)
        irr = compute_irr(cashflows, dates, self.cob)
        return irr
    
    def get_combined_symbol_irr(self, listSymbols: list):
        trans = self.transactions[self.transactions['Symbol'].isin(listSymbols)]
        cashflows = trans['Amount ($)'].tolist()
        dates = trans['Run Date'].tolist()
        current_value = self.position.loc[self.position['Symbol'].isin(listSymbols), 'Current Value'].sum()
        cashflows.append(current_value)
        dates.append(self.cob)
        irr = compute_irr(cashflows, dates, self.cob)
        return irr
    
    def get_stock_symbol_list(self):
        symbols = self.individualTransactions['Symbol'].unique()
        stockSymbols = [
            sym for sym in symbols
            if sym not in self.cashSymbols
            and not sym.startswith('91')
            and sym not in self.otherSymbols
            and sym not in self.contributionSymbols
        ]
        return stockSymbols
    
    def get_bond_symbol_list(self):
        symbols = self.individualTransactions['Symbol'].unique()
        bondSymbols = [
            sym for sym in symbols
            if  sym.startswith('91')
        ]
        return bondSymbols
    
    def get_401k_description_list(self):
        descriptions = self.pensionTransactions['Description'].unique()
        return descriptions
    

## Check get_individual_account_summary

In [29]:
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'
transactions = load_transaction(data_folder_path, transaction_file_pattern)
position = load_position(data_folder_path, position_file_pattern)
portfolio = Portfolio(transactions, position)

The latest transaction date is 2025-08-05


In [15]:
result = portfolio.get_individual_account_summary()
display_percentage(result,['Percentage','IRR'])

,Type,Value,Percentage,IRR,Weighted Avg Holding Period
0,bond,746743.50,62.47%,4.37%,6.113349
1,stock,423303.14,35.41%,16.28%,12.168792
2,cash,25294.19,2.12%,nan%,10.239369
3,Total,1195340.83,100.00%,8.67%,6.653133


## Check get_all_stock_summary

In [ ]:
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'
transactions = load_transaction(data_folder_path, transaction_file_pattern)
position = load_position(data_folder_path, position_file_pattern)
portfolio = Portfolio(transactions, position)

The latest transaction date is 2025-08-05


In [18]:
result = portfolio.get_all_stock_summary()
display_percentage(result,['Percentage','IRR'])

,Symbol,Current Value,Percentage,IRR,Weighted Avg Holding Period
21,Total,423303.14,100.00%,16.28%,12.168792
8,FXAIX,163865.08,38.71%,19.57%,9.521609
11,FSKAX,113381.14,26.78%,13.44%,11.276006
10,FSPSX,69497.82,16.42%,13.14%,13.441002
13,MSFT,15867.00,3.75%,16.98%,12.891512
0,AAPL,14233.47,3.36%,10.24%,34.997598
7,TSLA,6193.40,1.46%,32.63%,18.534063
9,MCD,6013.60,1.42%,14.87%,14.835822
14,BRKB,4661.80,1.10%,1.69%,9.600000
4,GOOGL,3888.10,0.92%,30.96%,29.867213


In [19]:
stockSymbols = portfolio.get_stock_symbol_list()
currentValueResult = portfolio.get_symbol_current_values(stockSymbols)
currentValueResult

,Symbol,Current Value,Percentage
0,AAPL,14233.47,0.033625
1,SBUX,2707.05,0.006395
2,JPM,2915.42,0.006887
3,AXP,2981.20,0.007043
4,GOOGL,3888.10,0.009185
5,AMZN,2136.90,0.005048
6,NKE,2230.05,0.005268
7,TSLA,6193.40,0.014631
8,FXAIX,163865.08,0.387110
9,MCD,6013.60,0.014206


In [21]:
irrResult = portfolio.get_symbol_irrs(stockSymbols)
irrResult

,Symbol,IRR
0,AAPL,0.102383
1,SBUX,0.065357
2,JPM,0.323283
3,AXP,0.245696
4,GOOGL,0.309569
5,AMZN,0.329863
6,NKE,-0.071231
7,TSLA,0.326300
8,FXAIX,0.195717
9,MCD,0.148729


In [22]:
holdingPeriod = portfolio.get_symbol_holding_period(stockSymbols)
holdingPeriod

,Symbol,Weighted Avg Holding Period
0,AAPL,34.997598
1,AMZN,29.900000
2,AXP,35.666667
3,BRKB,9.600000
4,COKE,13.218008
5,COST,0.933333
6,FSKAX,11.276006
7,FSPSX,13.441002
8,FXAIX,9.521609
9,GOOGL,29.867213


In [23]:
df = portfolio.individualTransactions[portfolio.individualTransactions['Amount ($)']<0].copy()

In [24]:
df['Days Held'] = (portfolio.cob - df['Run Date']).apply(lambda x: x.days)
df['Weight'] = df['Amount ($)'].abs()

# 分组计算加权平均
weighted_hold = (
    df.groupby('Symbol')
      .apply(lambda g: (g['Days Held'] * g['Weight']).sum() / g['Weight'].sum()/ 365, include_groups=False)
      .reset_index(name='Weighted Avg Holding Years')
)
# weighted_hold['Weighted Avg Holding Years'] = weighted_hold['Weighted Avg Holding Years'] / 365
weighted_hold

,Symbol,Weighted Avg Holding Years
0,912796ZV4,0.731507
1,912797GB7,1.328767
2,912797GN1,1.594521
3,912797GW1,0.961644
4,912797GZ4,1.594521
...,...,...
72,PG,0.076712
73,SBUX,2.399611
74,TSLA,1.523348
75,Transfer,1.283265


## Check get_bond_irr

In [ ]:
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'
transactions = load_transaction(data_folder_path, transaction_file_pattern)
position = load_position(data_folder_path, position_file_pattern)
portfolio = Portfolio(transactions, position)

The latest transaction date is 2025-08-05


In [34]:
result = portfolio.get_all_bond_summary(showExpiredBond=False)
display_percentage(result,['Percentage','IRR'])

,Symbol,Current Value,Percentage,IRR,Weighted Avg Holding Period
54,Total,746743.5,100.00%,4.37%,6.113349
46,912797PW1,149338.5,20.00%,3.71%,0.820074
50,912797QU4,149253.0,19.99%,3.37%,0.899616
48,912797MG9,99976.0,13.39%,4.18%,1.433333
49,912797QL4,99751.0,13.36%,3.88%,1.366667
51,912797QT7,99586.0,13.34%,3.41%,0.900000
53,912797QW0,99336.0,13.30%,0.20%,0.200000
52,912797NA1,49503.0,6.63%,3.14%,0.300000


## Check get 401k account summary

In [35]:
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'
transactions = load_transaction(data_folder_path, transaction_file_pattern)
position = load_position(data_folder_path, position_file_pattern)
portfolio = Portfolio(transactions, position)

The latest transaction date is 2025-08-05


In [40]:
portfolio.individualPosition['Description'].unique()

array(['FIDELITY 500 INDEX FUND',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 09/11/2025',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 09/16/2025',
       'FIDELITY TOTAL MARKET INDEX FUND',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 08/07/2025',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 08/26/2025',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 09/09/2025',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 09/30/2025',
       'FIDELITY INTERNATL INDEX FUND',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 10/30/2025',
       'HELD IN MONEY MARKET', 'MICROSOFT CORP', 'APPLE INC',
       'TESLA INC COM', 'MCDONALD S CORP',
       'BERKSHIRE HATHAWAY INC COM USD0.0033 CLASS B',
       'ALPHABET INC CAP STK CL A', 'AMERICAN EXPRESS CO COM USD0.20',
       'JPMORGAN CHASE &CO. COM', 'KRAFT HEINZ CO COM',
       'STARBUCKS CORP COM USD0.001', 'COCA COLA CONS INC COM',
       'NIKE INC CLASS B COM NPV', 'VERIZON COMMUNICATIONS INC',
       'AMAZ

In [39]:
portfolio.individualTransactions['Description'].unique()

array([' No Description', ' FIDELITY TREASURY MONEY MARKET FUND',
       ' APPLE INC', ' STARBUCKS CORP COM USD0.001',
       ' JPMORGAN CHASE &CO. COM', ' AMERICAN EXPRESS CO COM USD0.20',
       ' ALPHABET INC CAP STK CL A', ' AMAZON.COM INC',
       ' NIKE INC CLASS B COM NPV',
       ' UNITED STATES TREAS BILLS ZERO CPN 0.00000% 03/26/2024',
       ' UNITED STATES TREAS BILLS ZERO CPN 0.00000% 03/21/2024',
       ' UNITED STATES TREAS BILLS ZERO CPN 0.00000% 02/15/2024',
       ' UNITED STATES TREAS BILLS ZERO CPN 0.00000% 04/04/2024',
       ' UNITED STATES TREAS BILLS ZERO CPN 0.00000% 04/25/2024',
       ' TESLA INC COM', ' FIDELITY 500 INDEX FUND',
       ' UNITED STATES TREAS BILLS ZERO CPN 0.00000% 05/09/2024',
       ' UNITED STATES TREAS BILLS ZERO CPN 0.00000% 06/06/2024',
       ' MCDONALD S CORP',
       ' UNITED STATES TREAS BILLS ZERO CPN 0.00000% 07/11/2024',
       ' UNITED STATES TREAS BILLS ZERO CPN 0.00000% 05/30/2024',
       ' UNITED STATES TREAS BILLS ZERO CPN 